In [1]:
from gen_model import gen_model

In [9]:
uf, event_data = gen_model('libya_hotel_tweets.tsv')

2018-10-05 16:20:17,045 : load and clean dataset: libya_hotel_tweets.tsv
2018-10-05 16:20:17,204 : tweets processed: 26331, ignored: 2309, missing urls: 6341
2018-10-05 16:20:17,205 : create pairs (t, u) or (t, t') for each tweet t and url u or replied/retweeted tweet t'
2018-10-05 16:20:17,228 : total pairs: 20108, retweets: 11323, quotes: 0, replies: 289 (missing: 0)
2018-10-05 16:20:17,228 : applying union-find
2018-10-05 16:20:17,376 : total components: 2747


In [23]:
uf.components()

[{'560029594967744513', 'http://cur.lv/iaj1e'},
 {'560035788600000512',
  'https://twitter.com/News_World_/status/560035788600000512/photo/1'},
 {'560026145047842816',
  '560113155968622592',
  '560050011921477632',
  '557568838385295360',
  '560062773325873152',
  '560029343704154116',
  '560042572547371009',
  '560026215583850496',
  '560103595119366146',
  '559781954498080770',
  '560037081691725824',
  '560074811003928576',
  '560027208865951744',
  '560027349538701312',
  '560047714084610048',
  '560027226444296192',
  '560026986283036672',
  '560027751131127808',
  '560056946225475587',
  '560070877531086848',
  '560041697523269632',
  '560032939132477440',
  '560043882491031552',
  '560090951021260800',
  '559984770331836418',
  '560027543789920256',
  '560026422069059584',
  '560026966070272000',
  '560044543463415808',
  '560179249530015744',
  '560126669000687616',
  '560038724931956736',
  '560179549951246336',
  '560193189483925505',
  '560027526425112576',
  '5600468542271

In [24]:
from gensim.models import KeyedVectors

we = KeyedVectors.load_word2vec_format('/home/mquezada/anchor-text-twitter/data/ft_alltweets_model.vec')

/home/mquezada/anaconda3/envs/news-model/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/mquezada/anaconda3/envs/news-model/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
2018-10-08 12:33:41,063 : 'pattern' package not found; tag filters are not available for English
2018-10-08 12:33:41,066 : loading projection weights from /home/mquezada/anchor-text-twitter/data/ft_alltweets_model.vec
2018-10-08 12:34:47,548 : loaded (1076139, 100) matrix from /home/mquezada/anchor-text-twitter/data/ft_alltweets_model.vec


In [52]:
from pathlib import Path
import json
import spacy

nlp = spacy.load('en', disable=["tagger", "parser", "ner"])

In [53]:
event_data

{'560029596029292544': Tweet(tweet_id='560029596029292544', retweet_id='560025454057619456', quote_id='NULL', reply_id='NULL', short_urls=['http://t.co/dNG1ykqJi0'], expanded_urls={0: 'http://www.bbc.co.uk/news/world-africa-31001094'}, text='Gunmen possibly linked to Islamic State attack hotel popular with foreigners in Libyan capital Tripoli -officials http://t.co/dNG1ykqJi0', created_at='2015-01-27 11:00:35'),
 '560025454057619456': Tweet(tweet_id='560025454057619456', retweet_id='NULL', quote_id='NULL', reply_id='NULL', short_urls=['http://t.co/dNG1ykqJi0'], expanded_urls={0: 'http://www.bbc.co.uk/news/world-africa-31001094'}, text='Gunmen possibly linked to Islamic State attack hotel popular with foreigners in Libyan capital Tripoli -officials http://t.co/dNG1ykqJi0', created_at='2015-01-27 10:44:07'),
 '560029594967744513': Tweet(tweet_id='560029594967744513', retweet_id='NULL', quote_id='NULL', reply_id='NULL', short_urls=['http://t.co/lHyssteVh4'], expanded_urls={0: 'http://cur.

In [58]:
from collections import defaultdict
from tqdm import tqdm

docs = defaultdict(list)
docs_twids = defaultdict(list)

for c in tqdm(uf.components()):
    keys = []
    texts = []
    ids = []
    for e in c:
        if e.startswith('http'):
            keys.append(e)
        else:
            t = event_data[e]
            texts.append(t.text)
            ids.append(t.tweet_id)
    docs[tuple(keys)] = texts
    docs_twids[tuple(keys)] = ids

100%|██████████| 2747/2747 [00:00<00:00, 99176.69it/s]


In [55]:
docs

defaultdict(list,
            {('http://cur.lv/iaj1e',): ['Gunmen take hostages at luxury hotel in Libya http://t.co/lHyssteVh4'],
             ('https://twitter.com/News_World_/status/560035788600000512/photo/1',): ['#Car bomb explodes outside luxurious hotel in Libyas Tripoli: A car... http://t.co/dYcT89HcWd #Resistance #politics http://t.co/YTfRTBGFGz'],
             ('https://twitter.com/NewsOnTheMin/status/559994589906739200/photo/1',
              'http://hosted2.ap.org/APDEFAULT/3d281c11a96b4ad082fe88aa0db04305/Article_2015-01-27-ML-Libya/id-ff3332fe800848579c1505bdcdef0562',
              'http://deals.ebay.com/',
              'https://twitter.com/DoyleGlobal/status/560120714825461760/photo/1',
              'https://www.rt.com/news/226603-libya-tripoli-gunmen-seige/',
              'http://www.middleeasteye.net/news/gunmen-kill-guards-and-take-hostages-corinthia-hotel-libyas-tripoli-705958223',
              'https://twitter.com/SputnikInt/status/560032170111406080/photo/1',


In [59]:
docs_twids

defaultdict(list,
            {('http://cur.lv/iaj1e',): ['560029594967744513'],
             ('https://twitter.com/News_World_/status/560035788600000512/photo/1',): ['560035788600000512'],
             ('https://twitter.com/NewsOnTheMin/status/559994589906739200/photo/1',
              'http://hosted2.ap.org/APDEFAULT/3d281c11a96b4ad082fe88aa0db04305/Article_2015-01-27-ML-Libya/id-ff3332fe800848579c1505bdcdef0562',
              'http://deals.ebay.com/',
              'https://twitter.com/DoyleGlobal/status/560120714825461760/photo/1',
              'https://www.rt.com/news/226603-libya-tripoli-gunmen-seige/',
              'http://www.middleeasteye.net/news/gunmen-kill-guards-and-take-hostages-corinthia-hotel-libyas-tripoli-705958223',
              'https://twitter.com/SputnikInt/status/560032170111406080/photo/1',
              'https://sputniknews.com/middleeast/201501271017397901/',
              'http://ww38.shellfc.com/',
              'http://deals.ebay.com/5001818620_5_Star_H

In [41]:
import numpy as np

av_docs = list()
vecs = dict()

for j, (doc, texts) in enumerate(docs.items()):    
    doc_vecs = []
    for text in nlp.pipe(texts, n_threads=8):
        text_vec = [we[token.lower_] for token in text if not token.like_url and token.lower_ in we]
        doc_vecs.extend(text_vec)
        
    if len(doc_vecs) > 0:
        avg_vec = np.array(doc_vecs).mean(axis=0)
        vecs[doc] = avg_vec
        av_docs.append(j)

In [60]:
base = 'data_local_events/'
v_file = 'libya_vectors.txt'
u_file = 'libya_urls.txt'
t_file = 'libya_texts.txt'
i_file = 'libya_ids.txt'

p1 = Path(base + v_file)
p2 = Path(base + u_file)
p3 = Path(base + t_file)
p4 = Path(base + i_file)

with p1.open('w') as f, p2.open('w') as g, p3.open('w') as k, p4.open('w') as l:
    for i, (idf, v) in enumerate(vecs.items()):
        f.write(f"{i + 1}" + "\t" + '\t'.join(str(vv) for vv in v) + "\n")
        for u in idf:
            g.write(f"{i + 1}" + "\t" + f"{u}" + "\n")
        texts = docs[idf]
        for text in texts:
            k.write(f"{i + 1}" + "\t" + f'"{text}"' + "\n")
        for id_ in docs_twids[idf]:
            l.write(f"{i + 1}" + "\t" + f'{id_}' + "\n")

In [1]:
len(vecs)

NameError: name 'vecs' is not defined